# OLAP Operationen für GBI Daten
* wir verwenden hier die Daten aus dem BW4 Curriculum
* Dateiformat csv
* getrennte Fakten für US und DE
* Jahre 2007 bis 2019
* Revenue nur in lokaler Währung

Faktentabelle DE einlesen
* Feld- und Dezimaltrenner werden übergeben
* Datum wir geparsed

In [ ]:
import pandas as pd

df_facts_de = pd.read_csv("SalesdataBW4DeV01.csv", header = 0,delimiter =';',decimal=',', parse_dates=['SalesDate'])
df_facts_de.head()

Faktentabelle US einlesen 

In [ ]:
df_facts_us = pd.read_csv("SalesdataBW4UsV01.csv", header = 0,delimiter =',',decimal='.', parse_dates=['SalesDate'])
df_facts_us.head()


Fakten zusammenführen

In [ ]:
df_facts = pd.concat([df_facts_de, df_facts_us])
len(df_facts.axes[0])

Dimension Customer einlesen

In [ ]:
df_cust = pd.read_csv("CustomerBW4V01.csv", header = 0,delimiter =';')
df_cust.head()

Inner Join Faktentabelle und Dimensionstabelle

In [ ]:
df_analysis = pd.merge(df_facts, df_cust, how='inner', left_on = 'Customer', right_on = 'Customer')
df_analysis.head()

Auswertung mit Aggregation eines Feldes

In [ ]:
import numpy as np
df_analysis.groupby('Country').agg({'SalesQuantity': np.sum, 'Revenue': np.sum})

Jahr ableiten

In [ ]:
df_analysis['Year'] =  df_analysis['SalesDate'].dt.year
df_analysis.groupby('Year').agg({'SalesQuantity': np.sum})

Aggregation mehrerer Felder; kann mit den Zahlen in Case A2 verglichen werden

In [ ]:
df_analysis.groupby(['Year', 'Country']).agg({ 'Revenue': np.sum})

Filter setzen

In [ ]:
df_analysis[df_analysis['Country']=='DE'].groupby('Year').agg({'Revenue': np.sum})

Drill Down, usw...

In [ ]:
df_analysis[df_analysis['Country']=='DE'].groupby(['Year','Salesorg']).agg({'Revenue': np.sum})